In [119]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import anthropic
import gradio as gr
import io
import sys
import subprocess

In [120]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-'):
    print('API key exists, good to go.')
else:
    print("API key doesn't exist, troubleshoot the problem")

API key exists, good to go.


In [121]:
openai = OpenAI()
GPT_MODEL = 'gpt-4o-mini'
anthropic = anthropic.Anthropic()
CLAUDE_MODEL = 'claude-3-5-sonnet-20240620'

system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. "
system_message += "#include all necesarry libaries or packages that are required for a C++ code."


def user_message(python_code):
    prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    prompt += python_code
    return prompt

In [122]:
def message_for(python_code):
    return [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message(python_code)}
    ]
    

In [124]:
def write_cpp_code_to_file(cpp_code):
    code = cpp_code.replace("```cpp\n", "").replace("```", "")
    with open('optimized.cpp', 'w') as f:
        f.write(code)
    

In [125]:
def stream_gpt_response(python_code):
    response = openai.chat.completions.create(
        model= GPT_MODEL,
        messages= message_for(python_code),
        stream= True
    )
    result = ""
    for chunk in response:
        stream = chunk.choices[0].delta.content or ""
        result += stream
    yield result.replace("```cpp\n", "").replace("```", "")
    
    write_cpp_code_to_file(result)


In [126]:
def stream_claude_response(python_code):
    response = anthropic.messages.stream(
        model= CLAUDE_MODEL,
        system= system_message,
        messages= [{"role": "user", "content": user_message(python_code)}],
        max_tokens= 200
    )
    result = ""
    with response as stream:
        for chunk in stream.text_stream:
            result += chunk
    yield result.replace("```cpp\n", "").replace("```", "")
    
    write_cpp_code_to_file(result)


In [127]:
py_code = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [128]:
exec(py_code)

Result: 3.141592658589
Execution Time: 21.734672 seconds


In [129]:
stream_gpt_response(py_code)

<generator object stream_gpt_response at 0x152240c80>

In [130]:
def select_streaming_model(model, python_code):
    if model == 'GPT':
        response = stream_gpt_response(python_code)
    elif model == 'Claude':
        response = stream_claude_response(python_code)
    else:
        raise ValueError("Unknown model")

    for streaming_text in response:
        yield streaming_text

In [ ]:
# with gr.Blocks() as ui:
#     with gr.Row():
#         python = gr.Textbox(label="Python code:", lines=10, value=py_code)
#         cpp = gr.Textbox(label="C++ code:", lines=10)
#     with gr.Row():
#         model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
#         convert = gr.Button("Convert code")

#     convert.click(select_streaming_model, inputs=[model, python], outputs=[cpp])

# ui.launch(inbrowser=True)


Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


In [132]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [133]:
def execute_cpp(code):
        write_cpp_code_to_file(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [134]:
with gr.Blocks() as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=py_code, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(select_streaming_model, inputs=[model, python], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.
